In [20]:
#!pip install tweepy

In [1]:
cd /home/jovyan/GA_DSI/Projects/capstone/

/home/jovyan/GA_DSI/Projects/capstone


In [2]:
import tweepy
import twitter
from twitter import OAuth 
from twitter import TwitterStream 
import csv
import pickle
import pandas as pd
from tqdm import tqdm
from time import sleep
import pymongo
import lib.mongo_utilities as mongo

## Data acquisition from twitter.

In [77]:
####input your credentials here
my_auth_dict={
               'CONSUMER_KEY':'vSDP5sxilw6chhuD2CwtEyOmw',
               'CONSUMER_SECRET':'burFncmKxLlZme9IHwukubgXSi8CqHrTzg7ui0x09M2tQ7nOUb',
               'ACCESS_TOKEN':'930880332827521024-isEVd6n420hBVlriG5eRQaFA0PIu6bD',
               'ACCESS_SECRET':'klQlKPoVHVBWAa2XL0pmM7DBzLBcb42sUPhlzN8hgugvU'
             }

# authentication for twitterstream
oauth = OAuth(my_auth_dict['ACCESS_TOKEN'],
              my_auth_dict['ACCESS_SECRET'],
              my_auth_dict['CONSUMER_KEY'],
              my_auth_dict['CONSUMER_SECRET'])

twitter_stream = TwitterStream(auth=oauth)

# authentication for tweepy.
consumer_key = my_auth_dict['CONSUMER_KEY']
consumer_secret = my_auth_dict['CONSUMER_SECRET']
access_token = my_auth_dict['ACCESS_TOKEN']
access_token_secret = my_auth_dict['ACCESS_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [11]:
# Getting extended text from user timeline.
# new_tweets = api.user_timeline(screen_name = screen_name,count=200, tweet_mode="extended")
# tweets = [[tweet.full_text] for tweet in new_tweets]

In [11]:
# airline_twit_handles = ['@VirginAmerica', '@united', '@SouthwestAir',
#                        '@JetBlue', '@USAirways', '@AmericanAir', '@Delta']

# hashtags: #unitedAIRLINES, #americanAIRLINES, #southWestAIRLINES, #virginAmerica
#            USAirways, #JetBlue, #Delta

In [ ]:
# The following method collects tweets for a give airline and stores in a csvfile.

In [7]:
def pull_tweets_store_csv(airline, filename, date, num_tweets):
    '''
    This method pulls tweets from twitter and saves them to a csv file.
    '''
    
    # Open/Create a file to append data
    csvFile = open(filename, 'a')

    #Use csv Writer
    csvWriter = csv.writer(csvFile)

    for tweet in tweepy.Cursor(api.search,q="#"+airline,count=num_tweets,
                               lang="en",
                               tweet_mode='extended',
                               since=date).items():
        
        print (tweet.created_at, tweet.full_text)
        csvWriter.writerow([airline, tweet.created_at, tweet.full_text.encode('utf-8')])
                                                                              

In [104]:
def tweepy_pull_tweets_store_db(airline, coll_ref, date, num_tweets):
    '''
    This method pulls tweets from twitter and saves them to mongo database.
    '''
    for tweet in tweepy.Cursor(api.search,
                               q = "#" + airline,
                               count = num_tweets,
                               lang="en",
                               tweet_mode = 'extended',
                               since=date).items():
        
        print (airline, tweet.created_at, tweet.full_text)
        
        # insert into database. 
        # coll_ref is handle to the collection in the database.
        coll_ref.insert_one({'airline': airline, 'date': tweet.created_at, 'text': tweet.full_text})


In [122]:
# pull 1000 tweets for each airline.
filename = "data/csvfiles/pulled_airline_tweets.csv"
date = "2017-12-05"
airline_name = ["UnitedAirlines", "AmericanAirlines", "SouthWestAirlines",
                "VirginAmerica", "USAirways", "JetBlue", "Delta"]
    
for name in tqdm(airline_name):
    pull_tweets_store_csv(name, filename, date, num_tweets=100)

  0%|          | 0/7 [00:00<?, ?it/s]

2017-12-12 23:21:54 When it comes to in flight entertainment, what do you want from the airline. You can't please all of the people all of the time but you sure can please none of the people some of the time. #AmericanAirlines #SouthwestAirlines #UnitedAirlines #JetBlue  https://t.co/LCJWRUaDit
2017-12-12 23:10:03 @AlonzoBodden @AlonzoBodden you are righter than ever with this old tweet, because #AmericanAirlines and #UnitedAirlines have banned Basic Economy passengers from using the bins, setting up even more conflict! https://t.co/ELyx2NK84p
2017-12-12 23:03:00 By banning Basic Economy ticket holders from using the overhead bins on #airliners, #AmericanAirlines and #UnitedAirlines are forcing flight attendants to become cops. https://t.co/R49n38FelY
2017-12-12 21:40:14 1960s United Airlines 'Fly The Friendly Skies' Scripto Butane Lighter https://t.co/Mnwyh0YZhK @EtsySellsVtg #scripto #etsyseller #shopetsy #vintage #vintagelighters #lighters #tobacciana @united #UnitedAirlines #smokin

2017-12-11 05:05:11 #Disappointed in @united’s #Website #failure - changing a #meal within the allotted time frame online, called to confirm but record was not updated. Staff would not make any effort to assist! #travel #unitedairlines #nomas
2017-12-11 04:26:24 RT @BrianGPowell: https://t.co/G9heShNmoe
#ProjectMockingBird has been a direct threat to #USA #Citizens
#UnitedAirLines{#UA} stock shorted…
2017-12-11 04:20:08 https://t.co/G9heShNmoe
#ProjectMockingBird has been a direct threat to #USA #Citizens
#UnitedAirLines{#UA} stock shorted thru #DeutscheBank{#DB} on #9ll
#Woolsey+#Rice+#Biden+#BO tried to kidnap #Turkish cleric so he couldn't testify on #9ll--they tried to blame on #Flynn
2017-12-11 03:58:30 RT @ariawebb19: non-refundable airline tickets are the most ridiculous concept I've ever been inconvenienced with #UnitedAirlines @united
2017-12-11 03:36:28 @JessicaClackum Agreed. Very frustrated. Thank you for the support at this trying time. #United #United sucks #UnitedAirline

2017-12-09 16:57:21 RT @UnitedFlyerHD: United B777-200ER N76010 lining up on Houston IAH 33L
#Avgeek #unitedairlines #boeing #Boeing777 https://t.co/VICSVgTfw1
2017-12-09 16:40:33 RT @UnitedFlyerHD: United B777-200ER N76010 lining up on Houston IAH 33L
#Avgeek #unitedairlines #boeing #Boeing777 https://t.co/VICSVgTfw1
2017-12-09 16:18:40 RT @nonrevnick: Love the colors that all the circuit breakers make! #unitedairlines #weareunited #pilotlife #ramplife #megaplane #denverint…
2017-12-09 16:17:56 RT @nonrevnick: Love the colors that all the circuit breakers make! #unitedairlines #weareunited #pilotlife #ramplife #megaplane #denverint…
2017-12-09 16:06:02 Had an economics professor who said it was ok to be a pig but not a hog. Guess the  #Airlines HOGS don’t make enough - $4.2 billion on baggage fees in 2017. #DeltaAirlines #AmericanAirlines #UnitedAirlines #baggage #Luggage #CarryOn https://t.co/UP7PwEGKPY
2017-12-09 14:55:48 #unitedAIRLINES  is the Cleveland Browns of airlines
2017-12-

2017-12-08 03:24:37 RT @UnitedFlyerHD: United B777-200ER N76010 lining up on Houston IAH 33L
#Avgeek #unitedairlines #boeing #Boeing777 https://t.co/Ml92zeGeQY
2017-12-08 03:23:39 RT @UnitedFlyerHD: United B777-200ER N76010 lining up on Houston IAH 33L
#Avgeek #unitedairlines #boeing #Boeing777 https://t.co/VICSVgTfw1
2017-12-08 03:23:07 RT @UnitedFlyerHD: United B777-200ER N76010 lining up on Houston IAH 33L
#Avgeek #unitedairlines #boeing #Boeing777 https://t.co/VICSVgTfw1
2017-12-08 03:22:28 United B777-200ER N76010 lining up on Houston IAH 33L
#Avgeek #unitedairlines #boeing #Boeing777 https://t.co/VICSVgTfw1
2017-12-08 03:21:20 Why do I continue to support this airline? #unitedAIRLINES #1Kflyer #millionmileflyer
2017-12-08 03:19:54 Remarkable ineptitude by #unitedAIRLINES today. Deicer didn’t work, mechanics fixed and then it ran out of gas. Three hours and counting and I will miss connection to Sydney. Have to spend a day in LA instead
2017-12-08 03:14:16 #unitedAIRLINES #makeitr

...

2017-12-06 23:02:45 Find out how #Sclerotherapy is most effective on our website! #Delta https://t.co/GQlESL0Rsw https://t.co/YQdhuIrLiy
2017-12-06 22:56:03 Still won't get me to ever fly #delta. https://t.co/uedyytPT96
2017-12-06 22:40:56 Another @Delta FA interview down! Hope I’m part of the #Delta family soon! #keepclimbing 🔺
2017-12-06 22:40:04 Why Everyone Loves the Delta In2ition Shower Head https://t.co/2oL927huBP #showerhead #delta
2017-12-06 22:37:57 Arrived into Indy 24 hours ago... Can I go back to France?!? #fly #travel #delta #travelbug #sky… https://t.co/6AclQmIKcq
2017-12-06 22:33:21 I’m traveling! Not going to bed. #Delta #Not #Spiritair What is  wrong with you people today? #pajamalife https://t.co/riR3SKuvr3
2017-12-06 22:18:35 #DeltaAirlines #SkyTeam #Delta #ONE #BusinessClass #Dinner flat bed seats #MCO #Orlando - GRU #SaoPaulo https://t.co/7DBCFFfmUt via @youtube
2017-12-06 22:14:30 @Google @amazon 

This is my complaint account.

Congratulations you made it. It wa

2017-12-06 15:12:09 @Delta Had great service from Shirley, working at the Delta Assistance desk in Chicago ORD next to Gate E7. I’m on the 1015 to JFK and she was fantastic, getting me a window seat, offering me a complimentary Starbucks coffee whilst I wait for her to organize my ticket. #delta
2017-12-06 15:06:56 RT @GorkaGorospe: Lesser Yellowlegs at #Ebro #Delta today was a new addition to my #BY #phonescoping https://t.co/A4PmtG7ixO
2017-12-06 14:57:06 RT @GoldboxATL: #Delta Air Lines flight makes emergency bathroom stop in #Montana https://t.co/XcJYCcaGPK
2017-12-06 14:43:07 McLean Institute Hosts #VirtualReality Experience
Public will get a glimpse of #technology work conducted in the #Delta https://t.co/FnYnAGpnxc https://t.co/l02XYQJyFc
2017-12-06 14:42:28 RT @GoldboxATL: #Delta Air Lines flight makes emergency bathroom stop in #Montana https://t.co/XcJYCcaGPK
2017-12-06 14:36:43 #Delta Air Lines flight makes emergency bathroom stop in #Montana https://t.co/XcJYCcaGPK
2017-12-

2017-12-06 01:12:16 #Young Lady #Rescued From #Ritualists In #Delta #State (Photos)  https://t.co/SCWrhSUary
2017-12-06 00:50:48 Delta Analytics - Data Professionals for the Social Good https://t.co/Ytrgf9OHib #delta #analytics https://t.co/8mjXWyz716
2017-12-06 00:44:02 Thinking of starting a @gofundme page so I can afford @Delta airfare to the @rosebowlgame #GoDawgs #RoseBowl #Delta
2017-12-06 00:42:17 Awesome early Xmas gift! Couldn’t wait until the 25th to open them. ( Plus @michele_blais couldn’t keep it a secret) 
#ProjectRock ##delta
@TheRock https://t.co/K50o10J2IZ
2017-12-06 00:30:46 Door-to-Door #Limo in #Abbotsford, #Vancouver, #Surrey, #Mission, #Chilliwcak, #Delta, #Langley. Arrive Relaxed and On - Time. https://t.co/DNZg7zRByN
2017-12-06 00:26:24 In partnership with the Delta Regional Foundation, @CA_DPC created the #Delta Leadership Program to support #leadership development within its #sacdelta community. Find out how you can get involved: https://t.co/Uaq17n7qVx. #cawa

100%|██████████| 7/7 [00:24<00:00,  3.53s/it]

2017-12-05 14:14:19 RT @DeltaRegional: #SmallBiz not only showcases #Delta #talent &amp; #innovation, but creates jobs &amp; drives #EconDev in #rural communities. Lea…
2017-12-05 13:54:15 RT @sylvainfaust: #CSeries at #Aeromexico - One More Picture
 #mexico #bombardier #boeing #delta #travel #aviation #airline https://t.co/r9…
2017-12-05 13:52:51 #CSeries at #Aeromexico - One More Picture
 #mexico #bombardier #boeing #delta #travel #aviation #airline https://t.co/r9KJTTCDKo
2017-12-05 13:43:09 RT @scuderiagta: Happy  #TurboTuesday 😎
#Lancia #Delta HF 4WD #Kankkunen #Piironen
#WRC #oldschool 1987

@LanciaClub https://t.co/P9uiB0qHli
2017-12-05 13:41:41 RT @RadioGrindulufm: #Terputar

• @AtthetopBand -Good Night, Good Bye
•  For Her Dreams - Disisi Hati @playlist_ent

#DELTA #DeretanLaguTer…
2017-12-05 13:29:01 RT @RadioGrindulufm: #Terputar

• @AtthetopBand -Good Night, Good Bye
•  For Her Dreams - Disisi Hati @playlist_ent

#DELTA #DeretanLaguTer…
2017-12-05 13:22:03 Super Moon over M

In [112]:
df = pd.read_csv("data/csvfiles/test1.csv", 
                 encoding='utf-8', 
                 header=None,
                 names=['airline', 'date', 'tweet'])

In [113]:
df.shape

(2904, 3)

In [116]:
df.tweet[0]

"b'RT @MADlitMelll: #unitedAIRLINES this is absurd \\\\U0001f624 https://t.co/eW9UYEqpW0'"

In [45]:
# Somehow not writing emojis correctly to file, or maybe reading incorrectly... explore.

In [ ]:
#*************************** Insert tweets into database. *******************************

In [106]:
mongo_IP = '34.213.243.241'
mongo_port = 27016
mongoDBname = "airline_database"
collec_name = "new_airline_tweets"

# Instantiate client to our Mongo Server
client = pymongo.MongoClient(mongo_IP,mongo_port)    
# get a handle to the twitter database
db_ref = client[mongoDBname]    
# get a reference to specified collection. If it does not exist, it will be created.
coll_ref = db_ref[collec_name]

In [111]:
mongo.mongoDB_get_DBnames(), mongo.mongoDB_get_collection_names(mongoDBname), coll_ref.count()

(['admin',
  'airline_database',
  'local',
  'test',
  'twit_database',
  'wiki_database'],
 ['new_airline_tweets', 'airline_tweets'],
 7061)

In [108]:
# Get tweets since this date.
airline_names = ["UnitedAirlines", "AmericanAirlines", "SouthWestAirlines",
                "VirginAmerica", "USAirways", "JetBlue", "Delta"]
date_since = "2017-12-27"

In [ ]:
# pull tweets and store in database.
for airline in airline_names:
    print(airline)
    tweepy_pull_tweets_store_db(airline, coll_ref, date_since, num_tweets = 100)

In [110]:
coll_ref.count()

7061

### Now let's read the stored tweets.

In [112]:
mongoDBname = "airline_database"
collec_name = "new_airline_tweets"

In [113]:
airline_tw_collec = mongo.mongoDB_read_collection(mongoDBname, collec_name)

In [114]:
type(airline_tw_collec)

list

In [115]:
df = pd.DataFrame(airline_tw_collec)
df.head(2)

,_id,airline,date,text
0,5a5535a91fac9004a2f202d6,UnitedAirlines,2018-01-09 21:19:36,Kudos to the #unitedairlines staff for helping...
1,5a5535a91fac9004a2f202d7,UnitedAirlines,2018-01-09 21:06:39,And the painful experience with @united conti...


In [116]:
df.shape 

(7061, 4)

In [117]:
df.drop('_id', axis = 1,inplace=True)

In [119]:
df.shape 

(7061, 3)

In [120]:
df.head()

,airline,date,text
0,UnitedAirlines,2018-01-09 21:19:36,Kudos to the #unitedairlines staff for helping...
1,UnitedAirlines,2018-01-09 21:06:39,And the painful experience with @united conti...
2,UnitedAirlines,2018-01-09 21:02:07,@united is the shittiest airline company I hav...
3,UnitedAirlines,2018-01-09 20:56:00,Oh joy. You came home. #lostluggage #unitedair...
4,UnitedAirlines,2018-01-09 20:49:48,RT @officiParas: #UnitedAirlines Offers Custom...


In [121]:
df.airline.unique()

array(['UnitedAirlines', 'AmericanAirlines', 'SouthWestAirlines',
       'VirginAmerica', 'USAirways', 'JetBlue', 'Delta'], dtype=object)

In [122]:
filename = 'data/pulled_tweets/PT_all_airlines_df'
filehandler = open(filename,"wb")
pickle.dump(df, filehandler)